<a href="https://colab.research.google.com/github/Snowdenstyll/keno/blob/main/jupyterNotebooks/Keno_Lottery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [39]:
df = pd.read_csv("2024.csv")

In [40]:
df.head()
df.tail()
df.info()
df.describe()
df.drop(['PlayDate', 'AP'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   PlayDate  136 non-null    object
 1   AP        136 non-null    object
 2   N01       136 non-null    int64 
 3   N02       136 non-null    int64 
 4   N03       136 non-null    int64 
 5   N04       136 non-null    int64 
 6   N05       136 non-null    int64 
 7   N06       136 non-null    int64 
 8   N07       136 non-null    int64 
 9   N08       136 non-null    int64 
 10  N09       136 non-null    int64 
 11  N10       136 non-null    int64 
 12  N11       136 non-null    int64 
 13  N12       136 non-null    int64 
 14  N13       136 non-null    int64 
 15  N14       136 non-null    int64 
 16  N15       136 non-null    int64 
 17  N16       136 non-null    int64 
 18  N17       136 non-null    int64 
 19  N18       136 non-null    int64 
 20  N19       136 non-null    int64 
 21  N20       136 no

In [41]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [42]:
# All our games
number_of_rows = df.values.shape[0]
# Amount of games we need to take into consideration for prediction
window_length = 10
number_of_features = df.values.shape[1]

X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)


In [43]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [44]:
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
batch_size = 60

# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))

In [45]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.95, random_state=95)

# Train the model
history = model.fit(X, y, batch_size=batch_size, epochs=1000, verbose=2)
#history = model.fit(X_train, y_train, batch_size=100, epochs=600, verbose=2)

Epoch 1/1000
3/3 - 14s - loss: 1.0189 - accuracy: 0.0556 - 14s/epoch - 5s/step
Epoch 2/1000
3/3 - 0s - loss: 1.0137 - accuracy: 0.0714 - 74ms/epoch - 25ms/step
Epoch 3/1000
3/3 - 0s - loss: 1.0113 - accuracy: 0.0794 - 74ms/epoch - 25ms/step
Epoch 4/1000
3/3 - 0s - loss: 1.0091 - accuracy: 0.1270 - 73ms/epoch - 24ms/step
Epoch 5/1000
3/3 - 0s - loss: 1.0056 - accuracy: 0.1032 - 66ms/epoch - 22ms/step
Epoch 6/1000
3/3 - 0s - loss: 1.0015 - accuracy: 0.1032 - 66ms/epoch - 22ms/step
Epoch 7/1000
3/3 - 0s - loss: 1.0001 - accuracy: 0.0952 - 70ms/epoch - 23ms/step
Epoch 8/1000
3/3 - 0s - loss: 0.9968 - accuracy: 0.1270 - 69ms/epoch - 23ms/step
Epoch 9/1000
3/3 - 0s - loss: 0.9936 - accuracy: 0.1032 - 66ms/epoch - 22ms/step
Epoch 10/1000
3/3 - 0s - loss: 0.9891 - accuracy: 0.0952 - 68ms/epoch - 23ms/step
Epoch 11/1000
3/3 - 0s - loss: 0.9862 - accuracy: 0.1032 - 69ms/epoch - 23ms/step
Epoch 12/1000
3/3 - 0s - loss: 0.9828 - accuracy: 0.1111 - 68ms/epoch - 23ms/step
Epoch 13/1000
3/3 - 0s - lo

In [46]:
loss, accuracy = model.evaluate(X, y, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

4/4 [==============================] - 2s 10ms/step - loss: 0.0093 - accuracy: 0.9048
Test Loss: 0.00929989479482174, Test Accuracy: 0.9047619104385376
4/4 [==============================] - 0s 14ms/step - loss: 0.0093 - accuracy: 0.9000
Test Loss: 0.009329449385404587, Test Accuracy: 0.8999999761581421


In [47]:
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)

In [48]:
df1 = df.copy()
to_predict = df1.tail(window_length)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

<ipython-input-48-efb52fef0daf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)


,N01,N02,N03,N04,N05,N06,N07,N08,N09,N10,N11,N12,N13,N14,N15,N16,N17,N18,N19,N20
126,6,23,28,29,31,32,36,37,39,40,44,49,51,52,54,60,61,62,63,68
127,3,13,14,21,26,28,30,31,34,35,39,40,47,53,54,58,60,62,65,68
128,3,5,11,16,17,18,20,23,30,35,38,39,42,43,49,57,59,61,62,67
129,4,11,12,20,22,23,31,36,41,44,45,49,52,54,58,59,60,61,64,68
130,4,12,24,26,29,31,33,39,40,42,50,51,52,53,54,58,59,63,68,70
131,1,4,8,9,15,16,21,26,28,30,38,39,43,47,48,55,61,63,64,66
132,2,12,13,14,17,21,25,28,29,30,34,40,47,56,58,64,65,66,67,68
133,4,9,10,11,13,16,18,21,22,34,35,36,39,44,49,50,53,58,60,66
134,2,3,7,8,10,11,13,22,24,25,31,37,38,39,46,49,55,58,59,62


In [49]:
prediction = np.array(df.tail(1))

x_next = scaler.transform(to_predict)
y_pred = model.predict(np.array([x_next]))

temp = np.array(np.unique(scaler.inverse_transform(y_pred).astype(int)[0]))
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)

uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[uniq_idx[counts>=2] ]  # <--- Get duplicates

top_5_indices = np.argsort(y_pred[0], axis=0)[::-1][:10]
top_5_numbers = []

winning_indices = np.where(np.isin(prediction[0], duplicates))[0]

# Print the winning indices
print("The winning indices:", winning_indices)

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices:", top_5_indices)
print("The top 5 numbers:", top_5_numbers)

print("The predicted numbers for the lottery game are:", temp)
print("The actual numbers in the last lottery game were:", prediction[0])
print("\n")
print("winners:", duplicates)
print(len(duplicates)/20)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 4s 4s/step
The winning indices: [ 0  1  3  4  8 11 19]
The top 5 indices: [15 19 18 16 14 12  0 10 11 17]
The top 5 numbers: [55, 66, 64, 58, 50, 42, 2, 35, 36, 59]
The predicted numbers for the lottery game are: [ 1  2  4  8  9 11 16 21 25 26 33 36 40 41 47 51 54 56 61 66]
The actual numbers in the last lottery game were: [ 2  4  6  9 11 13 19 23 26 30 35 36 42 44 50 55 58 59 64 66]


winners: [ 2  4  9 11 26 36 66]
0.35


# Predict the next set of numbers

In [50]:
next = df.copy()
next = next.tail(window_length)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))
prediction = scaler.inverse_transform(y_next_pred).astype(int)
print("The predicted numbers for the lottery game:", temp)
print("\n")
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]
top_5_numbers = []
for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)
print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)


1/1 [==============================] - 2s 2s/step
The predicted numbers for the lottery game: [ 1  2  4  8  9 11 16 21 25 26 33 36 40 41 47 51 54 56 61 66]


The top 5 indices [19 14  8 13 15]
The top 5 numbers: [70, 49, 27, 44, 50]
winning indices [ 0  1  3  4  8 11 19]
winning numbers [1, 2, 8, 11, 27, 35, 70]


In [53]:
next = df.copy()
next = next.tail(1)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))

# Get the actual values of the top 10 predictions
prediction = scaler.inverse_transform(y_next_pred).astype(int)

# Get the indices of the top 10 predictions (change axis=0 if predictions are along a different axis)
top_10_indices = np.argsort(y_pred[0], axis=0)[::-1][:10]
top_5_indices = np.argsort(y_pred[0], axis=0)[::-1][:5]

top_10_numbers = []
top_5_numbers = []

# index [0,2,4] usually are correct
# Loop through each index in top_10_indices
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_5_numbers.append(number)

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)
print("\n")

print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("The predicted numbers:", prediction)


1/1 [==============================] - 0s 21ms/step
winning indices [ 0  1  3  4  8 11 19]
winning numbers [3, 6, 13, 17, 29, 39, 67]


The top 5 indices [15 19 18 16 14]
The top 5 numbers: [53, 67, 63, 56, 49]


The top 10 indices [15 19 18 16 14 12  0 10 11 17]
The top 10 numbers: [53, 67, 63, 56, 49, 43, 3, 36, 39, 59]


The predicted numbers: [[ 3  6 10 13 17 19 23 26 29 33 36 39 43 46 49 53 56 59 63 67]]
